# Sentiment_analysis_using_CNN for bank_review_project

In [1]:
from __future__ import division, print_function
from gensim import models
from keras.callbacks import ModelCheckpoint
from keras.layers import Dense, Dropout, Reshape, Flatten, concatenate, Input, Conv1D, GlobalMaxPooling1D, Embedding
from keras.layers.recurrent import LSTM
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import os
import collections
import re
import string

Using TensorFlow backend.


In [2]:
import pandas as pd
TEXT = pd.read_excel("C:\\Users\\RAMKUMAR\\Desktop\\BankReviews.xlsx")
TEXT

,Date,Stars,Reviews,BankName
0,2017-04-10,5,"Great job, Wyndham Capital! Each person was pr...",Wyndham Capital Mortgage
1,2017-02-10,5,Matthew Richardson is professional and helpful...,Wyndham Capital Mortgage
2,2017-08-21,5,We had a past experience with Wyndham Mortgage...,Wyndham Capital Mortgage
3,2017-12-17,5,We have been dealing with Brad Thomka from the...,Wyndham Capital Mortgage
4,2016-05-27,5,I can't express how grateful I am for the supp...,Wyndham Capital Mortgage
...,...,...,...,...
500,2016-02-06,1,\r\nI never write reviews but had to this time...,North American Savings Bank
501,2016-07-25,1,\r\nIt all started when Bob G ran a credit che...,North American Savings Bank
502,2017-09-27,1,\r\nWhat a horrible experience. We have excell...,North American Savings Bank
503,2017-12-24,1,"\r\nRep was extremely professional, friendly, ...",North American Savings Bank


In [3]:
TEXT.Stars.unique()

array([5, 1], dtype=int64)

In [4]:
TEXT.shape

(505, 4)

In [5]:
pos = []
neg = []
for l in TEXT.Stars:
    if l == 1:
        pos.append(1)
        neg.append(5)
    elif l == 5:
        pos.append(5)
        neg.append(1)

In [6]:
TEXT['Pos']= pos
TEXT['Neg']= neg

In [7]:
TEXT.head()

,Date,Stars,Reviews,BankName,Pos,Neg
0,2017-04-10,5,"Great job, Wyndham Capital! Each person was pr...",Wyndham Capital Mortgage,5,1
1,2017-02-10,5,Matthew Richardson is professional and helpful...,Wyndham Capital Mortgage,5,1
2,2017-08-21,5,We had a past experience with Wyndham Mortgage...,Wyndham Capital Mortgage,5,1
3,2017-12-17,5,We have been dealing with Brad Thomka from the...,Wyndham Capital Mortgage,5,1
4,2016-05-27,5,I can't express how grateful I am for the supp...,Wyndham Capital Mortgage,5,1


In [8]:
TEXT

,Date,Stars,Reviews,BankName,Pos,Neg
0,2017-04-10,5,"Great job, Wyndham Capital! Each person was pr...",Wyndham Capital Mortgage,5,1
1,2017-02-10,5,Matthew Richardson is professional and helpful...,Wyndham Capital Mortgage,5,1
2,2017-08-21,5,We had a past experience with Wyndham Mortgage...,Wyndham Capital Mortgage,5,1
3,2017-12-17,5,We have been dealing with Brad Thomka from the...,Wyndham Capital Mortgage,5,1
4,2016-05-27,5,I can't express how grateful I am for the supp...,Wyndham Capital Mortgage,5,1
...,...,...,...,...,...,...
500,2016-02-06,1,\r\nI never write reviews but had to this time...,North American Savings Bank,1,5
501,2016-07-25,1,\r\nIt all started when Bob G ran a credit che...,North American Savings Bank,1,5
502,2017-09-27,1,\r\nWhat a horrible experience. We have excell...,North American Savings Bank,1,5
503,2017-12-24,1,"\r\nRep was extremely professional, friendly, ...",North American Savings Bank,1,5


# Clean data

In [9]:
#def remove_punct(text):
   # text_nopunct = ''
    #text_nopunct = re.sub('['+string.punctuation+']', '', text)
    #return text_nopunct
#data['Text_Clean'] = data['Text'].apply(lambda x: remove_punct(x))

In [10]:
import re
def remove_punct(text):
    text_nopunct = ''
    text_nopunct = re.sub('['+string.punctuation+']', '', text)
    return text_nopunct
TEXT['Text_Clean'] = TEXT['Reviews'].apply(lambda x: remove_punct(x))

In [11]:
from nltk import word_tokenize, WordNetLemmatizer
tokens = [word_tokenize(sen) for sen in TEXT.Text_Clean]

In [12]:
def lower_token(tokens): 
    return [w.lower() for w in tokens]    
    
lower_tokens = [lower_token(token) for token in tokens]

In [13]:
from nltk.corpus import stopwords
stoplist = stopwords.words('english')

In [14]:
def remove_stop_words(tokens): 
    return [word for word in tokens if word not in stoplist]

In [15]:
filtered_words = [remove_stop_words(sen) for sen in lower_tokens]

In [16]:
result = [' '.join(sen) for sen in filtered_words]

In [17]:
TEXT['Text_Final'] = result

In [18]:
TEXT['tokens'] = filtered_words

In [19]:
TEXT = TEXT[['Text_Final', 'tokens', 'Stars', 'Pos', 'Neg']]

In [20]:
TEXT[:4]

,Text_Final,tokens,Stars,Pos,Neg
0,great job wyndham capital person professional ...,"[great, job, wyndham, capital, person, profess...",5,5,1
1,matthew richardson professional helpful helped...,"[matthew, richardson, professional, helpful, h...",5,5,1
2,past experience wyndham mortgage would without...,"[past, experience, wyndham, mortgage, would, w...",5,5,1
3,dealing brad thomka beginning started stressfu...,"[dealing, brad, thomka, beginning, started, st...",5,5,1


In [21]:
TEXT

,Text_Final,tokens,Stars,Pos,Neg
0,great job wyndham capital person professional ...,"[great, job, wyndham, capital, person, profess...",5,5,1
1,matthew richardson professional helpful helped...,"[matthew, richardson, professional, helpful, h...",5,5,1
2,past experience wyndham mortgage would without...,"[past, experience, wyndham, mortgage, would, w...",5,5,1
3,dealing brad thomka beginning started stressfu...,"[dealing, brad, thomka, beginning, started, st...",5,5,1
4,cant express grateful support zach provided fa...,"[cant, express, grateful, support, zach, provi...",5,5,1
...,...,...,...,...,...
500,never write reviews time prevent anyone making...,"[never, write, reviews, time, prevent, anyone,...",1,1,5
501,started bob g ran credit check without knowled...,"[started, bob, g, ran, credit, check, without,...",1,1,5
502,horrible experience excellent credit plenty in...,"[horrible, experience, excellent, credit, plen...",1,1,5
503,rep extremely professional friendly helpful fi...,"[rep, extremely, professional, friendly, helpf...",1,1,5


# Split data into test and train

In [22]:
TEXT_train, TEXT_test = train_test_split(TEXT, test_size=0.10, random_state=42)

In [23]:
all_training_words = [word for tokens in TEXT_train["tokens"] for word in tokens]
training_sentence_lengths = [len(tokens) for tokens in TEXT_train["tokens"]]
TRAINING_VOCAB = sorted(list(set(all_training_words)))
print("%s words total, with a vocabulary size of %s" % (len(all_training_words), len(TRAINING_VOCAB)))
print("Max sentence length is %s" % max(training_sentence_lengths))

17551 words total, with a vocabulary size of 2530
Max sentence length is 256


In [24]:
all_test_words = [word for tokens in TEXT_test["tokens"] for word in tokens]
test_sentence_lengths = [len(tokens) for tokens in TEXT_test["tokens"]]
TEST_VOCAB = sorted(list(set(all_test_words)))
print("%s words total, with a vocabulary size of %s" % (len(all_test_words), len(TEST_VOCAB)))
print("Max sentence length is %s" % max(test_sentence_lengths))

2091 words total, with a vocabulary size of 941
Max sentence length is 209


# Load Google News Word2Vec model

In [25]:
word2vec_path = 'C:\\Users\\RAMKUMAR\\Desktop\\GoogleNews-vectors-negative300.bin'
word2vec = models.KeyedVectors.load_word2vec_format(word2vec_path, binary=True)

In [27]:
def get_average_word2vec(tokens_list, vector, generate_missing=False, k=300):
    if len(tokens_list)<1:
        return np.zeros(k)
    if generate_missing:
        vectorized = [vector[word] if word in vector else np.random.rand(k) for word in tokens_list]
    else:
        vectorized = [vector[word] if word in vector else np.zeros(k) for word in tokens_list]
    length = len(vectorized)
    summed = np.sum(vectorized, axis=0)
    averaged = np.divide(summed, length)
    return averaged

def get_word2vec_embeddings(vectors, clean_comments, generate_missing=False):
    embeddings = clean_comments['tokens'].apply(lambda x: get_average_word2vec(x, vectors, 
                                                                                generate_missing=generate_missing))
    return list(embeddings)

# Get Embeddings

In [28]:
training_embeddings = get_word2vec_embeddings(word2vec, TEXT_train, generate_missing=True)

In [29]:
MAX_SEQUENCE_LENGTH = 50
EMBEDDING_DIM = 300

# Tokenize and Pad sequences

In [30]:
tokenizer = Tokenizer(num_words=len(TRAINING_VOCAB), lower=True, char_level=False)
tokenizer.fit_on_texts(TEXT_train["Text_Final"].tolist())
training_sequences = tokenizer.texts_to_sequences(TEXT_train["Text_Final"].tolist())

train_word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(train_word_index))

Found 2530 unique tokens.


In [32]:
train_cnn_data = pad_sequences(training_sequences, maxlen=MAX_SEQUENCE_LENGTH)

In [33]:
train_embedding_weights = np.zeros((len(train_word_index)+1, EMBEDDING_DIM))
for word,index in train_word_index.items():
    train_embedding_weights[index,:] = word2vec[word] if word in word2vec else np.random.rand(EMBEDDING_DIM)
print(train_embedding_weights.shape)

(2531, 300)


In [34]:
test_sequences = tokenizer.texts_to_sequences(TEXT_test["Text_Final"].tolist())
test_cnn_data = pad_sequences(test_sequences, maxlen=MAX_SEQUENCE_LENGTH)

# Define CNN

In [35]:
def ConvNet(embeddings, max_sequence_length, num_words, embedding_dim, labels_index):
    
    embedding_layer = Embedding(num_words,
                            embedding_dim,
                            weights=[embeddings],
                            input_length=max_sequence_length,
                            trainable=False)
    
    sequence_input = Input(shape=(max_sequence_length,), dtype='int32')
    embedded_sequences = embedding_layer(sequence_input)

    convs = []
    filter_sizes = [2,3,4,5,6]

    for filter_size in filter_sizes:
        l_conv = Conv1D(filters=200, kernel_size=filter_size, activation='relu')(embedded_sequences)
        l_pool = GlobalMaxPooling1D()(l_conv)
        convs.append(l_pool)


    l_merge = concatenate(convs, axis=1)

    x = Dropout(0.1)(l_merge)  
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.2)(x)
    preds = Dense(labels_index, activation='sigmoid')(x)

    model = Model(sequence_input, preds)
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['acc'])
    model.summary()
    return model

In [36]:
label_names = ['Pos', 'Neg']

In [37]:
y_train = TEXT_train[label_names].values

In [39]:
x_train = train_cnn_data
y_tr = y_train

In [40]:
model = ConvNet(train_embedding_weights, MAX_SEQUENCE_LENGTH, len(train_word_index)+1, EMBEDDING_DIM, 
                len(list(label_names)))

W1219 11:48:47.246377  8764 deprecation.py:323] From C:\Users\RAMKUMAR\Documents\Python Scripts\lib\site-packages\tensorflow\python\ops\nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 50)           0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 50, 300)      759300      input_1[0][0]                    
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 49, 200)      120200      embedding_1[0][0]                
__________________________________________________________________________________________________
conv1d_2 (Conv1D)               (None, 48, 200)      180200      embedding_1[0][0]                
____________________________________________________________________________________________

# Train CNN

In [41]:
num_epochs = 3
batch_size = 34

In [42]:
hist = model.fit(x_train, y_tr, epochs=num_epochs, validation_split=0.1, shuffle=True, batch_size=batch_size)

W1219 11:49:58.603224  8764 deprecation_wrapper.py:119] From C:\Users\RAMKUMAR\Documents\Python Scripts\lib\site-packages\keras\backend\tensorflow_backend.py:422: The name tf.global_variables is deprecated. Please use tf.compat.v1.global_variables instead.



Train on 408 samples, validate on 46 samples
Epoch 1/3
408/408 [==============================] - 4s 9ms/step - loss: -52.0728 - acc: 0.4988 - val_loss: -151.8697 - val_acc: 0.5000
Epoch 2/3
408/408 [==============================] - 2s 4ms/step - loss: -341.0621 - acc: 0.5000 - val_loss: -679.3349 - val_acc: 0.5000
Epoch 3/3
408/408 [==============================] - 2s 4ms/step - loss: -1172.8564 - acc: 0.5000 - val_loss: -2003.3302 - val_acc: 0.5000


# Test CNN

In [43]:
predictions = model.predict(test_cnn_data, batch_size=1024, verbose=1)

51/51 [==============================] - 0s 4ms/step


In [44]:
Stars = [5, 1]

In [45]:
prediction_Stars=[]
for p in predictions:
    prediction_Stars.append(Stars[np.argmax(p)])

In [46]:
sum(TEXT_test.Stars==prediction_Stars)/len(prediction_Stars)

0.8627450980392157

In [47]:
TEXT_test.Stars.value_counts()

5    44
1     7
Name: Stars, dtype: int64